<a href="https://colab.research.google.com/github/Monserrat555/proyecto-1-de-mon/blob/main/Proyecto1_mon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Proyecto 1 de Tania Monserrat Gopar Iniesta
# Importaciones
import numpy as np
import pandas as pd
from google.colab import drive
# Definimos la clase Grafica
class Grafica:
    """Clase para hacer una grafica."""
    def __init__(self, nodos, arcos):
        self.matriz_adyacencia = np.zeros((len(nodos), len(nodos)))
        for x in arcos:
            self.matriz_adyacencia[x[0], x[1]] = 1

    def __str__(self):
        return str(self.matriz_adyacencia)
# Definimos de la clase Red
class Red(Grafica):
    """La Clase Red va a heredar de Grafica y calcular PageRank."""
    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)
        self.matriz_transicion = np.zeros_like(self.matriz_adyacencia)

    def calcular_matriz_transicion(self):
        """Calcula la matriz de transición."""
        for i in range(len(self.matriz_adyacencia)):
            suma_fila = self.matriz_adyacencia[i, :].sum()
            if suma_fila > 0:
                self.matriz_transicion[i, :] = self.matriz_adyacencia[i, :] / suma_fila
            else:
                self.matriz_transicion[i, :] = np.zeros(len(self.matriz_adyacencia))

    def calcular_pagerank(self, damping_factor=0.85, tolerancia=1e-6):
        """Aquí Calcula el PageRank con el damping factor."""
        n = len(self.matriz_adyacencia)
        self.normalizar_matriz()
        U = np.ones((n, n)) / n
        P = damping_factor * self.matriz_transicion + (1 - damping_factor) * U
        pr_vector = np.ones((1, n)) / n
        pr_vector_1 = pr_vector @ P

        while np.linalg.norm(pr_vector - pr_vector_1) > tolerancia:
            pr_vector = pr_vector_1
            pr_vector_1 = pr_vector @ P

        return pr_vector_1.flatten()

    def calcular_pagerank_ru(self, indices_ru, damping_factor=0.85, tolerancia=1e-6):
        """Aquí Calcula el PageRank modificado para páginas que terminan con .ru."""
        n = len(self.matriz_adyacencia)
        s = len(indices_ru)
        pr_vector = np.zeros((1, n))
        pr_vector[0, indices_ru] = 1 / s

        self.normalizar_matriz()
        U = np.ones((n, n)) / n
        P = damping_factor * self.matriz_transicion + (1 - damping_factor) * U
        pr_vector_1 = pr_vector @ P

        while np.linalg.norm(pr_vector - pr_vector_1) > tolerancia:
            pr_vector = pr_vector_1
            pr_vector_1 = pr_vector @ P

        return pr_vector_1.flatten()

def construir_red(ruta_archivo):
    """Aquí Construye un objeto Red con un archivo de Excel."""
    df = pd.read_excel(ruta_archivo)
    Nodos = list(range(len(df["Index"])))
    Arcos = []
    for i, citados in enumerate(df["Cited by"]):
        for citado in str(citados).split(','):
            if citado.strip().isdigit():
                Arcos.append((int(citado.strip()) - 1, i))

    return Red(Nodos, Arcos), df

if __name__ == "__main__":
    # Aquí está el Google Drive
    drive.mount('/content/drive', force_remount=True)
    ruta_excel = "/content/Web.xlsx"

    red, df = construir_red(ruta_excel)

    # Pregunta 1:
    resultado_pagerank = red.calcular_pagerank()
    pagina_mas_importante = np.argmax(resultado_pagerank) + 1  #sumo 1 porque porque los índices epiezan en 0

    print("Resultados de PageRank estándar (Pregunta 1):", resultado_pagerank)
    print("Página más importante (Pregunta 1):", pagina_mas_importante)

    # Pregunta 2:
    if "Website" in df.columns:
        indices_ru = [i for i, url in enumerate(df["Website"]) if str(url).endswith('.ru')]

        if indices_ru:
            resultado_pagerank_ru = red.calcular_pagerank_ru(indices_ru)
            pagina_mas_importante_ru = np.argmax(resultado_pagerank_ru) + 1

            print("\nResultados de PageRank con páginas .ru (Pregunta 2):", resultado_pagerank_ru)
            print("Página más importante con .ru (Pregunta 2):", pagina_mas_importante_ru)
        else:
            print("\nNo hay páginas que terminen en .ru con esos datos (Pregunta 2).")
    else:
        print("\nLa columna 'Website' no existe en los datos (Pregunta 2).")

    # Pregunta 3:
    valores_d = [0.5, 0.85, 1]
    print("\nResultados para diferentes valores de d (Pregunta 3):")
    for d in valores_d:
        try:
            resultado_pagerank_modificado = red.calcular_pagerank(d=d)
            print(f"\nd={d}")
            print("Vector de PageRank:", resultado_pagerank_modificado)
            print("Página más importante:", np.argmax(resultado_pagerank_modificado) + 1)
        except Exception as e:
            print(f"Error al calcular PageRank con d={d}: {str(e)}")